# Scheduling Optimization

Scheduling is a fundamental problem that arises in various contexts, from university course timetables to employee shifts and beyond. At its core, scheduling involves allocating resources to activities over time, ensuring that all constraints are met and objectives are optimized. This process requires careful consideration of several factors, including the availability of resources, the needs of the activities, and any specific constraints that must be adhered to.

In scheduling optimization, we aim to find the most efficient allocation of resources that meets all requirements while optimizing a specific goal. This goal can vary depending on the context, such as minimizing total operation time, maximizing resource utilization, or balancing workloads evenly.

The process involves defining variables to represent decisions (e.g., assigning a course to a time slot), constraints to ensure all requirements are met (e.g., no overlapping classes for a teacher), and an objective function that quantifies the goal we're trying to achieve. Solvers like Gurobi then use mathematical algorithms to find the best possible solution that satisfies all constraints and optimizes the objective function.

## Project Scenario: University Course Scheduling

In this project, we tackle the challenge of scheduling university courses. The goal is to assign each course to a specific room and time slot in a way that avoids conflicts and makes the best use of available resources. This involves several key considerations:

Room Capacity: Each course must be placed in a room that can hold all enrolled students.

Instructor and Student Schedules: We need to ensure that instructors aren't scheduled to teach two courses at the same time and that students can attend all their courses without any overlap.

Room Availability: Courses should be scheduled in available rooms during times when those rooms are not already in use.

Optimization Goals: Our main objectives are to minimize scheduling conflicts and to use rooms as efficiently as possible.

To solve this problem, we'll use mathematical modeling to represent our scheduling constraints and objectives. Then, we'll apply an optimization solver to find the best possible schedule that meets all our criteria.

### Mathematical Model for University Course Scheduling

The objective of our university course scheduling model is to assign courses to rooms and time slots in a way that accommodates all enrolled students, avoids instructor and student schedule conflicts, and optimizes room usage. The model is formulated as follows:

#### Sets and Indices
- $C$: Set of courses, indexed by $c$.
- $R$: Set of rooms, indexed by $r$.
- $T$: Set of time slots, indexed by $t$.

#### Parameters
- $S_c$: Number of students enrolled in course $c$.
- $C_r$: Capacity of room $r$.
- $A_{c,t}$: Availability of course $c$ at time $t$, where $A_{c,t} = 1$ if course $c$ can be scheduled at time $t$, and 0 otherwise.

#### Decision Variables
- $x_{c,r,t}$: Binary variable that equals 1 if course $c$ is assigned to room $r$ at time $t$, and 0 otherwise.

#### Objective Function
$$\text{Maximize} \quad Z = \sum_{c \in C} \sum_{r \in R} \sum_{t \in T} x_{c,r,t}$$
The objective function aims to maximize the number of course assignments, promoting efficient use of available resources.

#### Constraints

1. **Room Capacity Constraint**:
$$\sum_{c \in C} S_c \cdot x_{c,r,t} \leq C_r, \quad \forall r \in R, \forall t \in T$$
This constraint ensures that the number of students enrolled in a course does not exceed the capacity of the assigned room.

2. **Course Scheduling Constraint**:
$$\sum_{r \in R} \sum_{t \in T} x_{c,r,t} = 1, \quad \forall c \in C$$
Each course must be scheduled exactly once.

3. **Instructor Conflict Constraint**:
$$\sum_{c \in C} \sum_{r \in R} x_{c,r,t} \leq 1, \quad \forall t \in T, \text{ for each instructor}$$
Instructors cannot teach more than one course at the same time.

4. **Student Schedule Conflict Constraint**:
$$\sum_{c \in C} x_{c,r,t} \leq 1, \quad \forall t \in T, \text{ for each student}$$
Students cannot be required to attend more than one course at the same time.

5. **Room and Time Availability Constraint**:
$$x_{c,r,t} \leq A_{c,t}, \quad \forall c \in C, \forall r \in R, \forall t \in T$$
Courses can only be scheduled in available rooms and time slots.

#### Binary Variable Constraint
$$x_{c,r,t} \in \{0,1\}, \quad \forall c \in C, \forall r \in R, \forall t \in T$$

This mathematical model provides a comprehensive framework for addressing the university course scheduling problem, ensuring that all courses are assigned to appropriate rooms and times without exceeding room capacities or causing scheduling conflicts.


In [1]:
import gurobipy as gp
from gurobipy import GRB

# Create a new model
model = gp.Model("University Course Scheduling")

# Sample Data 
C = ['Course1', 'Course2', 'Course3']  # Courses
R = ['Room1', 'Room2']  # Rooms
T = ['Time1', 'Time2', 'Time3']  # Time slots
S = {'Course1': 30, 'Course2': 40, 'Course3': 35}  # Students enrolled
C_r = {'Room1': 50, 'Room2': 40}  # Room capacities
P = {('Course1', 'Room1'): 1, ('Course2', 'Room2'): 1, ('Course3', 'Room1'): 0}  # Specific room requirements
lambda_weight = 0.5

# Variables
x = model.addVars(C, R, T, vtype=GRB.BINARY, name="x")

# Objective
model.setObjective(
    gp.quicksum(x[c, r, t] for c in C for r in R for t in T) -
    lambda_weight * gp.quicksum(x[c1, r1, t] * x[c2, r2, t] for c1 in C for c2 in C for t in T for r1 in R for r2 in R if c1 != c2),
    GRB.MINIMIZE
)

# Constraints
# Room Capacity
for c in C:
    for r in R:
        model.addConstr(gp.quicksum(S[c] * x[c, r, t] for t in T) <= C_r[r])

# Instructor Availability
# Assuming instructor data is available, constraints would be added similarly

# Time Slot Availability
for t in T:
    model.addConstr(gp.quicksum(x[c, r, t] for c in C for r in R) <= 1)

# Room Utilization
for r in R:
    model.addConstr(gp.quicksum(x[c, r, t] for c in C for t in T) >= 1)

# Specific Room Requirements
for c, r in P.keys():
    for t in T:
        model.addConstr(x[c, r, t] <= P[(c, r)])

# Optimize model
model.optimize()

# Print solution
for v in model.getVars():
    if v.x > 0:
        print(f'{v.varName}: {v.x}')

# Optimization Report
print(f'Optimal Objective: {model.objVal}')
print(f'Status: {model.status}')


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 18 columns and 63 nonzeros
Model fingerprint: 0xd9271149
Model has 36 quadratic objective terms
Variable types: 0 continuous, 18 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 2.0000000
Presolve removed 10 rows and 3 columns
Presolve time: 0.01s
Presolved: 34 rows, 39 columns, 117 nonzeros
Variable types: 0 continuous, 39 integer (39 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept